## **1- Importing Libraries**

In [7]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON



## **2- API Key & Channels ID's**

In [8]:
api_key = ''


In [9]:
channel_ids = ['UC7cs8q-gJRlGwj4A8OmCmXg', 'UCiT9RITQ9PW6BhXK0y2jaeg', 'UCJQJAI7IjbLcpsjWdSzYz0Q', 
                'UC1FksTDit4ij9ogFAUC9xxg', 'UCteRPiisgIoHtMgqHegpWAQ', 'UCh9nVJoWXmFb7sLApWGcLPQ',
                'UCe8377g-WtGma8H_qXaHOlQ', 'UCITfscBBm40O9MgtqiQC00w', 'UCDybamfye5An6p-j1t2YMsg',
                'UCtoNXlIegvxkvf5Ji8S57Ag', 'UCw_LFe2pS8x3NyipGNJgeEA', 'UCnz-ZXXER4jOvuED5trXfEA',
                'UCc6lpSsVek7NNwzcdKiArkw', 'UCMPsUECkDUtzWJ2KtlUVL4A', 'UCLLw7jmFsvfIVaUFsLs8mlQ',
                'UCIchWfEE4FevYhRh9HXsSbw', 'UCe4c5YQfFsCQVMsOnJOiTIw', 'UCeEcSgRzYFuVt-2Yk1ULdhQ',
                'UCNU_lfiiWBdtULKOw6X0Dig', 'UCcfngi7_ASuo5jdWX0bNauQ']

In [10]:
api_service_name = "youtube"
api_version = "v3"
youtube = build(api_service_name, api_version, developerKey =  api_key)

## **3- Channel Stats**

In [11]:
def get_channel_stats(youtube, channel_ids):
    
    all_data = []   
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
        data = {
            'Channel_Name': item['snippet']['title'],
            'Subscribers': item['statistics']['subscriberCount'],
            'Views': item['statistics']['viewCount'],
            'Total_Videos': item['statistics']['videoCount'],
            'Channel_Id': item['contentDetails']['relatedPlaylists']['uploads']
        } 
        all_data.append(data)    
    return pd.DataFrame(all_data)

In [12]:
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

,Channel_Name,Subscribers,Views,Total_Videos,Channel_Id
0,edrea,4080,189769,10,UUe8377g-WtGma8H_qXaHOlQ
1,Lore So What,13400,635609,93,UUtoNXlIegvxkvf5Ji8S57Ag
2,Mohamed Al Assaal - اتعلم مع العسال,413000,16197737,504,UUIchWfEE4FevYhRh9HXsSbw
3,Ranesh Guruparan,3660,112949,64,UUc6lpSsVek7NNwzcdKiArkw
4,Ken Jee,248000,8334564,280,UUiT9RITQ9PW6BhXK0y2jaeg
5,Learn with Lukas,16600,785705,38,UUw_LFe2pS8x3NyipGNJgeEA
6,Justin Shin,21400,1309119,37,UUe4c5YQfFsCQVMsOnJOiTIw
7,Thu Vu data analytics,160000,5103158,73,UUJQJAI7IjbLcpsjWdSzYz0Q
8,How to Power BI,147000,8257853,296,UUcfngi7_ASuo5jdWX0bNauQ
9,Alex The Analyst,544000,22041518,230,UU7cs8q-gJRlGwj4A8OmCmXg


## **4- Videos Details & ID's**

In [13]:
def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                    part='contentDetails',
                    playlistId = playlist_id,
                    maxResults = 50,
                    pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [14]:
def get_video_details(youtube, video_ids):

    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt', 'categoryId', 'defaultAudioLanguage', 'thumbnails'],
                            'statistics': ['viewCount', 'likeCount','dislikeCount', 'commentCount'],
                            'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = 0

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)

In [15]:
import re
import pandas as pd

def clean_text(text):
    # Remove non-alphanumeric characters and emojis
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

def get_comments_in_videos(youtube, video_ids):
    comments = []

    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()

            for item in response['items']:
                comment_data = item['snippet']['topLevelComment']['snippet']
                comment_text = comment_data['textOriginal']
                cleaned_comment = clean_text(comment_text)
                
                comment = {
                    'video_id': video_id,  
                    'comment_id': comment_data['authorChannelId']['value'],
                    'author': comment_data['authorDisplayName'],
                    'comment': cleaned_comment,
                    'like_count': comment_data['likeCount'],
                    'published_at': comment_data['publishedAt'],
                    'total_replies': item['snippet']['totalReplyCount']
                }
                comments.append(comment)

        except Exception as e:
            # When an error occurs, most likely because comments are disabled on a video
            if 'commentsDisabled' in str(e):
                print('Comments disabled for video ' + video_id)
                comment = {
                    'video_id': video_id,
                    'comment_id': '',
                    'author': '',
                    'comment': 'Disabled',
                    'like_count': 0,
                    'published_at': '',
                    'total_replies': 0
                }
                comments.append(comment)
            else:
                print('Could not get comments for video ' + video_id)
                print('Error:', e)

    return pd.DataFrame(comments)




## **4.1- Sundas Khalid Channel**

In [16]:
SundasKhalid_id = "UUteRPiisgIoHtMgqHegpWAQ"

# Get video IDs
SundasKhalid_video_ids = get_video_ids(youtube, SundasKhalid_id)

# Get video details
SundasKhalid_video_df = get_video_details(youtube, SundasKhalid_video_ids)
SundasKhalid_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,l14K2EnD548,Sundas Khalid,AI Will Replace Tech Jobs: From ex-FAANG Softw...,50 job interview questions & answers 👉🏼 https:...,"[data science, self-taugh data scientist, tech...",2023-08-07T14:10:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/l1...,4324,149,0,30,PT30M46S,hd,true
1,7ssLi7Ll0I0,Sundas Khalid,How Much Money I Made as Data Engineer? (3 yea...,Resume & Cover Letter template (free) 👉🏼 https...,"[data science, data scientist, self-taugh data...",2023-07-28T14:10:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/7s...,16491,296,0,30,PT6M14S,hd,true
2,xr68cbOxvBs,Sundas Khalid,How to learn Python FAST with ChatGPT and Bard?,Try Quadratic for FREE 👉🏼 https://QuadraticHQ....,"[data science, data scientist, self-taugh data...",2023-07-10T14:10:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/xr...,334755,3954,0,105,PT12M6S,hd,true
3,mLP4kdk3DoI,Sundas Khalid,Will AI Replace Data Scientists?,Excel graphs template (free) 👉🏼 https://clickh...,"[data science, data scientist, self-taugh data...",2023-06-29T14:12:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/mL...,18049,551,0,70,PT8M37S,hd,false
4,znouY2A61WI,Sundas Khalid,How to code Python FAST for Data Analysis: Bar...,Click to read full AI Trend Report (FREE) 👉🏼 h...,"[data science, data scientist, self-taugh data...",2023-06-16T14:10:01Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/zn...,23034,471,0,39,PT9M24S,hd,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,dz9ufVkTq5w,Sundas Khalid,Highly recommend reading ‘The Simple Path to W...,Highly recommend reading ‘The Simple Path to W...,0,2020-10-28T01:35:46Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/dz...,6920,144,0,5,PT16S,hd,false
104,K0yr0ZQ1_js,Sundas Khalid,How I Got into FAANG (Data Science),Hello friends!\n\nToday I’m talking about tips...,0,2020-09-27T16:00:10Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/K0...,15028,493,0,58,PT22M8S,hd,true
105,Jwc2SPl7v3Y,Sundas Khalid,Want to be a Data Scientist? This is the First...,"In this video, I talk about the first thing yo...","[data science, women in tech, data scientist, ...",2020-09-08T15:00:11Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/Jw...,26438,1145,0,50,PT4M11S,hd,true
106,aZiQCz31J4E,Sundas Khalid,First Day at Google - New York Noogler New Hir...,Sharing my first day Google experience at the ...,"[First Day at Google, Googler, Noogler, Women ...",2020-08-21T15:00:01Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/aZ...,57924,1105,0,91,PT20M14S,hd,true


In [17]:
# SundasKhalid_comment_df = get_comments_in_videos(youtube, SundasKhalid_video_ids)
# SundasKhalid_comment_df

In [18]:
# # file path to save the CSV file
# stats_csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Sundus Khaled/SundasKhalid_channel_stats.csv'
# # Save the channel_stats DataFrame to a CSV file
# SundasKhalid_video_df.to_csv(stats_csv_file_path, index=False)

# comments_csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Sundus Khaled/SundasKhalid_comments.csv'
# # Save the comments DataFrame to a CSV file
# SundasKhalid_comments_df.to_csv(comments_csv_file_path, index=False)

## **4.2- Annamonova Channel**

In [19]:
Annamonova_id = "UUMPsUECkDUtzWJ2KtlUVL4A"
                
# Get video IDs
Annamonova_video_ids = get_video_ids(youtube, Annamonova_id)

# Get video details
Annamonova_video_df = get_video_details(youtube, Annamonova_video_ids)
Annamonova_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,JB3VXcv4naM,Annamonova,"6 Insights I got by attending Meetups, that in...",Sharing with you top insights for your career ...,"[meetup, data analyst meetup, data analytics, ...",2023-08-02T12:30:17Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/JB...,125,13,0,1,PT12M32S,hd,false
1,N0cdUkVQ45w,Annamonova,Learning Data Analytics and Data Science throu...,the sense of data analysis and data science in...,"[data analyst, data analytics, data analyst jo...",2023-04-29T11:00:11Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/N0...,515,40,0,4,PT3M45S,hd,false
2,ifKzVI4CEfs,Annamonova,"Upwork Tutorial for beginners: profile tips, c...",Do you want to become a freelancer? Upwork is ...,"[upwork, upwork tutorial for beginners, upwork...",2023-03-25T12:15:05Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/if...,401,22,0,4,PT22M49S,hd,false
3,k_JGEW95YNo,Annamonova,Data Engineer tasks and job #dataengineer,,"[data engineer, data engineering, data science...",2023-03-22T02:11:34Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/k_...,2304,97,0,5,PT14S,hd,false
4,wdAOROQZNVE,Annamonova,That is Data Analyst,,"[data analyst, data analytics, data analyst jo...",2023-03-18T18:01:30Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/wd...,4482,314,0,8,PT26S,hd,false
5,KZnVVbV8QrI,Annamonova,"Insights from Q&A Data Analysis: Internships, ...","Answering your Questions about Data Analysis, ...","[data analyst, freelancing, how to start freel...",2023-03-17T14:00:14Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/KZ...,844,69,0,6,PT20M3S,hd,false
6,JRmuncIkDbw,Annamonova,TOP 3 MISTAKES ON INTERVIEW: Data Analyst inte...,#dataanalytics #datascience #interview,"[interview mistakes, interview tips, data anal...",2023-03-15T17:19:21Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/JR...,4769,244,0,4,PT30S,hd,false
7,YUwEzIIlGQM,Annamonova,"Unlocking the Secrets of Data Careers: skills,...",It's the ultimate guide to Data Careers! Data ...,"[data career paths, data careers, data analyst...",2023-03-09T13:30:20Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/YU...,733,40,0,3,PT15M38S,hd,false
8,p-yMe5Uw2yQ,Annamonova,Don't Miss This Major Tech News,Tech news is here! Today I want to discuss som...,"[tech news, technews, tech news briefing, bitc...",2023-02-20T14:30:16Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/p-...,209,15,0,3,PT4M45S,hd,false
9,DMLFDyXIMdc,Annamonova,Die Hard 3 problem or Data Analyst interview a...,#dataanalytics #datascience #interview,"[data analyst interview, data analyst intervie...",2023-02-17T17:08:59Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/DM...,8134,435,0,12,PT36S,hd,false


In [20]:
# Annamonova_comment_df = get_comments_in_videos(youtube, Annamonova_video_ids)
# Annamonova_comment_df

In [21]:
# # file path to save the CSV file
# csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Annamonova/Annamonova_channel_stats.csv'
# # Save the channel_stats DataFrame to a CSV file
# Annamonova_video_df.to_csv(csv_file_path, index=False)

# # file path to save the CSV file
# csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Annamonova/Annamonova_comments.csv'
# # Save the comments DataFrame to a CSV file
# Annamonova_comments_df.to_csv(csv_file_path, index=False)

## **4.3- Luke Barousse Channel**

In [22]:
LukeBarousse_id = "UULLw7jmFsvfIVaUFsLs8mlQ"
                
# Get video IDs
LukeBarousse_video_ids = get_video_ids(youtube, LukeBarousse_id)

# Get video details
LukeBarousse_video_df = get_video_details(youtube, LukeBarousse_video_ids)
LukeBarousse_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,hoTS5pIKgPU,Luke Barousse,The End of Data Analysts?!? (ChatGPT's Code In...,📊 Where I recommend learning data analysis 👉🏼 ...,"[data viz by luke, business intelligence, data...",2023-07-26T21:53:06Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/ho...,187534,3392,0,257,PT10M39S,hd,false
1,WSkcsrFCCXM,Luke Barousse,How I use ChatGPT as a Data Analyst,📊 Where I recommend learning data analysis 👉🏼 ...,"[data viz by luke, business intelligence, data...",2023-07-07T20:22:48Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/WS...,196185,3185,0,159,PT13M41S,hd,false
2,uyCXLr7cgrs,Luke Barousse,ChatGPT vs. Bard for Data Analysts,ChatGPT 👉🏼 https://chat.openai.com/\nBard 👉🏼 h...,"[data viz by luke, business intelligence, data...",2023-05-20T19:11:04Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/uy...,65285,1345,0,115,PT9M30S,hd,false
3,r9imv1z82jQ,Luke Barousse,Luke Barousse Live Stream,Full Video Here 👉🏼 youtu.be/XXXXXX\n\nCourses ...,"[data viz by luke, business intelligence, data...",2023-05-12T13:06:55Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/r9...,0,0,0,0,P0D,sd,false
4,Ss9SvcRORsk,Luke Barousse,Google’s BI vs. Advanced Data Analytics Certif...,📊 Business Intelligence Certificate 👉🏼 https:...,"[data viz by luke, business intelligence, data...",2023-04-13T01:38:59Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Ss...,124642,5617,0,252,PT10M31S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,FQiehwXLKV4,Luke Barousse,Conditional Format Tables in Tableau (Like Exc...,Tableau Fundamentals (DataCamp) 👉🏼 https://luk...,"[Tableau, Excel, Conditional, Conditionally, F...",2020-09-11T21:15:22Z,26,en-US,{'default': {'url': 'https://i.ytimg.com/vi/FQ...,9051,144,0,9,PT5M25S,hd,false
134,IdZuTave8QY,Luke Barousse,Create Stacked Bar Chart (and any other visual...,Tableau Fundamentals (DataCamp) 👉🏼 https://luk...,"[Tableau, Show Me, Stacked Bar Chart, Bar Char...",2020-09-11T08:59:20Z,26,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Id...,2157,82,0,8,PT6M11S,hd,false
135,W_5x9mURZe4,Luke Barousse,Dimensions Vs Measures (Blue Vs Green Data) -...,Tableau Fundamentals (DataCamp) 👉🏼 https://luk...,"[Tableau, Tableau Desktop, Dimensions, Measure...",2020-09-09T23:37:15Z,26,en-US,{'default': {'url': 'https://i.ytimg.com/vi/W_...,3056,131,0,10,PT11M18S,hd,false
136,sgS4TauqhJM,Luke Barousse,How To Use Tableau Desktop Controls - Tableau ...,Tableau Fundamentals (DataCamp) 👉🏼 https://luk...,"[Tableau, How To, How-To, Dashboard, Sheet, Ro...",2020-09-01T20:40:00Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/sg...,4436,168,0,10,PT9M13S,hd,false


In [23]:
# LukeBarousse_comment_df = get_comments_in_videos(youtube, LukeBarousse_video_ids)
# LukeBarousse_comment_df

In [24]:
# # file path to save the CSV file
# csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Luke Barousse/LukeBarousse_channel_stats.csv'
# # Save the channel_stats DataFrame to a CSV file
# LukeBarousse_video_df.to_csv(csv_file_path, index=False)

# # file path to save the CSV file
# csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Luke Barousse/LukeBarousse_comments.csv'
# # Save the comments DataFrame to a CSV file
# LukeBarousse_comments_df.to_csv(csv_file_path, index=False)

## **4.4- Edrea Channel**

In [25]:
edrea_id = "UUe8377g-WtGma8H_qXaHOlQ"

# Get video IDs
edrea_video_ids = get_video_ids(youtube, edrea_id)

# Get video details
edrea_video_df = get_video_details(youtube, edrea_video_ids)
edrea_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,UU-fxxQT2d8,edrea,I Moved to San Francisco Alone at 21,Surprise! I moved to San Francisco! I'll be ma...,"[san francisco, san francisco vlog, living in ...",2023-07-30T19:18:07Z,24,en,{'default': {'url': 'https://i.ytimg.com/vi/UU...,1567,91,0,29,PT5M35S,hd,false
1,_tOOc-_YZF8,edrea,PARENTS RATE MY COLLEGE MEALS,Watch me put my college culinary skills to the...,"[college, uc berkeley, uc berkeley vlog, colle...",2023-04-27T16:02:49Z,24,en,{'default': {'url': 'https://i.ytimg.com/vi/_t...,1399,82,0,26,PT5M56S,hd,false
2,qSbv-c8o_6c,edrea,I applied to 158 data science internships. Her...,"Hey, it’s Edrea! Here’s my advice on the data ...","[data science internship, applying to data sci...",2023-02-10T20:30:00Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/qS...,6782,552,0,35,PT7M29S,hd,false
3,hREfleo04GE,edrea,Day in the Life of a Data Science Student at U...,Come along w/ me on a day in my undergrad life...,"[data science student, berkeley data science, ...",2022-10-04T16:08:56Z,28,0,{'default': {'url': 'https://i.ytimg.com/vi/hR...,117378,2530,0,158,PT4M12S,hd,false
4,8INJaa_m4Po,edrea,Day in the Life of a Data Science Intern in Sa...,Follow me to work at my data science internshi...,"[data science, data science intern, day in the...",2022-08-11T23:07:59Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/8I...,52639,1452,0,100,PT4M46S,hd,false
5,zuJDkwEhA-E,edrea,"All 104 Movies I Watched In 2021, reviewed in ...",A brief summary/review of all 104 Movies I wat...,"[every movie i watched, movies, films, 104 mov...",2022-04-09T21:27:39Z,1,en,{'default': {'url': 'https://i.ytimg.com/vi/zu...,2493,93,0,20,PT20M43S,hd,false
6,WAQbsOtimGI,edrea,It's ok to be less than perfect.,Shoutout to the guy who stared at me filming i...,"[perfectionism, perfectionism and procrastinat...",2021-08-23T17:17:44Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/WA...,3017,125,0,39,PT9M8S,hd,false
7,m4h4p6BPpTs,edrea,1 Second Every Day of 2020,"Though 2020 was a wild ride for all of us, we ...",0,2021-01-07T11:22:58Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/m4...,1539,56,0,5,PT6M36S,hd,false
8,2wwua8bc_nc,edrea,1 second every day of 2019,"Here's to 2019. To all the great days, to all ...",0,2020-01-04T11:26:41Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/2w...,1598,37,0,3,PT7M20S,hd,false
9,zoaVTGF23bc,edrea,365 Seconds of 2018,Edrea's 2018 – One second at a time.,0,2019-01-02T09:35:31Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/zo...,1433,27,0,4,PT6M43S,hd,false


In [26]:
# edrea_comment_df = get_comments_in_videos(youtube, edrea_video_ids)
# edrea_comment_df

In [27]:
# # file path to save the CSV file
# csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project//Edrea/edrea_channel_stats.csv'
# # Save the channel_stats DataFrame to a CSV file
# edrea_video_df.to_csv(csv_file_path, index=False)

# # file path to save the CSV file
# csv_file_path = '/Users/godzilla/Desktop/Selected Topics-2/Project/Edrea/edrea_comments.csv'
# # Save the comments DataFrame to a CSV file
# edrea_comments_df.to_csv(csv_file_path, index=False)

## **4.5- Ranesh Guruparan Channel**

In [28]:
RaneshGuruparan_id = "UUc6lpSsVek7NNwzcdKiArkw"
                
# Get video IDs
RaneshGuruparan_video_ids = get_video_ids(youtube, RaneshGuruparan_id)

# Get video details
RaneshGuruparan_video_df = get_video_details(youtube, RaneshGuruparan_video_ids)
RaneshGuruparan_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,lxr9g6dGefs,Ranesh Guruparan,Advanced SQL Interview Questions for Data Anal...,"Hey guys, there is this Leetcode alternative f...",0,2023-08-08T04:00:32Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/lx...,215,12,0,0,PT3M54S,hd,false
1,ZqQmrd-uvIk,Ranesh Guruparan,Maximize Your Productivity with an Automated C...,Full Video Link: https://youtu.be/m5vG7PNXdYY,0,2023-08-07T04:00:13Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Zq...,90,10,0,0,PT57S,hd,false
2,NwLpvJlyXw4,Ranesh Guruparan,How To Boost Your Productivity Through Systems...,Full Video Link: https://youtu.be/m5vG7PNXdYY,0,2023-08-06T18:00:08Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Nw...,109,6,0,1,PT32S,hd,false
3,tgWpQV75nWw,Ranesh Guruparan,Study Smarter With These Techniques,Full Video Link: https://youtu.be/m5vG7PNXdYY,0,2023-08-06T17:35:13Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/tg...,136,11,0,3,PT56S,hd,false
4,m5vG7PNXdYY,Ranesh Guruparan,The Secret to Sustainable Productivity,I've been getting a few questions on productiv...,0,2023-08-06T04:00:18Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/m5...,457,16,0,12,PT6M34S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,GeVU8-UaF3o,Ranesh Guruparan,finals week in the life of a data science stud...,Inside the week of a data science senior 📚📖📝\n...,0,2023-05-05T04:03:02Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Ge...,139,8,0,2,PT7M13S,hd,false
60,NjQICOYA0nM,Ranesh Guruparan,24 hour data science case study // finals week...,Join me as I cram an entire case study in 24 h...,0,2023-05-03T18:26:00Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Nj...,208,14,0,0,PT6M16S,hd,false
61,60bHQH9rgZg,Ranesh Guruparan,study with me || multilayer feedforward neural...,Join me as I finish a project involving MLP's ...,0,2023-04-21T23:03:05Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/60...,215,6,0,0,PT11M42S,hd,false
62,qtEInAhwufs,Ranesh Guruparan,hello 👋,Website: https://raneshguruparan.com\nInstagra...,0,2023-04-19T01:44:17Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/qt...,233,15,0,0,PT1M20S,hd,false


In [29]:
# RaneshGuruparan_comment_df = get_comments_in_videos(youtube, RaneshGuruparan_video_ids)
# RaneshGuruparan_comment_df

In [30]:
# # file path to save the CSV file
# csv_file_path = 'RaneshGuruparan_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# RaneshGuruparan_video_df.to_csv(csv_file_path, index=False)

## **4.6- Mohamed Al Assaal Channel**

In [31]:
MohamedAlAssaal_id = "UUIchWfEE4FevYhRh9HXsSbw"
                
# Get video IDs
MohamedAlAssaal_video_ids = get_video_ids(youtube, MohamedAlAssaal_id)

# Get video details
MohamedAlAssaal_video_df = get_video_details(youtube, MohamedAlAssaal_video_ids)
MohamedAlAssaal_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,F_MqsTFg23A,Mohamed Al Assaal - اتعلم مع العسال,حلقه 15 - اعداد التقارير في تحليل البيانات - ...,تقدر تشترك في كورسات عن طريق الموقع بتاعي\nwww...,"[اتعلم مع العسال, العسال, الاكسل, اتعلم اكسل, ...",2023-08-05T17:00:15Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/F_...,2200,107,0,20,PT19M19S,hd,false
1,gMp6JPrveBk,Mohamed Al Assaal - اتعلم مع العسال,حلقه 14 الاعمده الشرطيه و معادلات Power Quer...,تقدر تشترك في كورسات عن طريق الموقع بتاعي\nwww...,"[اتعلم مع العسال, العسال, الاكسل, اتعلم اكسل, ...",2023-07-22T17:00:18Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/gM...,3313,126,0,17,PT39M9S,hd,false
2,ytmvL2FXqQM,Mohamed Al Assaal - اتعلم مع العسال,Multistreaming with Restream.io,Powered by Restream https://restream.io,0,2023-07-17T18:46:39Z,24,en-US,{'default': {'url': 'https://i.ytimg.com/vi/yt...,1071,24,0,0,PT15M36S,hd,false
3,Uy79TNKrBVs,Mohamed Al Assaal - اتعلم مع العسال,Multistreaming with Restream.io,Powered by Restream https://restream.io,0,2023-07-17T18:27:49Z,24,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Uy...,816,15,0,0,PT33S,hd,false
4,ngNmEpHSs0I,Mohamed Al Assaal - اتعلم مع العسال,حلقه 13 - التحليل بأستخدام الجداول المحوريه جز...,تقدر تشترك في كورسات عن طريق الموقع بتاعي\nwww...,"[اتعلم مع العسال, العسال, الاكسل, اتعلم اكسل, ...",2023-07-08T17:00:27Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/ng...,3113,112,0,27,PT21M9S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,KoE-9Ux0AXw,Mohamed Al Assaal - اتعلم مع العسال,Excel for Beginners حلقه 4 : اتعلم اكسل من الص...,النهارده هنتكلم علي عمليات حسابيه أساسيه في ال...,"[excel, اكسل, excel tutorial, microsoft excel ...",2021-02-23T18:00:04Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/Ko...,311483,8918,0,284,PT8M33S,hd,false
437,-fQ0dblhHUM,Mohamed Al Assaal - اتعلم مع العسال,Excel Sumifs - حلقه 3 : اتعلم اكسل من الصفر مع...,اهلا بكم في حلقه رقم 3 في اتعلم اكسل من الصفر ...,"[excel, اكسل, excel tutorial, microsoft excel ...",2021-02-20T18:00:00Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/-f...,411020,9815,0,422,PT3M52S,hd,false
438,XcuXLqLNJCY,Mohamed Al Assaal - اتعلم مع العسال,اتعلم اكسل من الصفر للمبتدئين - حلقه 2: ازاي ت...,أهلا بكم في حلقه جديده من تعليم ال Excel \nهنت...,"[اكسل, excel, microsoft excel, دورة اكسل, شرح ...",2021-02-16T18:00:18Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/Xc...,570626,12199,0,393,PT9M43S,hd,false
439,uRs8_EJqTFo,Mohamed Al Assaal - اتعلم مع العسال,اتعلم اكسل من الصفر للمبتدئين- الحلقه 1 - EXCE...,هنتعلم في الحلقه دي حاجات اساسيه و منها :\n- ا...,"[اكسل, Excel, تعليم, اتعلم, شرح الاكسل, الاكسل...",2021-02-13T18:00:11Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/uR...,1523374,41917,0,1168,PT15M2S,hd,false


In [32]:
# MohamedAlAssaal_comment_df = get_comments_in_videos(youtube, MohamedAlAssaal_video_ids)
# MohamedAlAssaal_comment_df

In [33]:
# # file path to save the CSV file
# csv_file_path = 'MohamedAlAssaal_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# MohamedAlAssaal_video_df.to_csv(csv_file_path, index=False)

## **4.7- Lore So What Channel**

In [34]:
LoreSoWhat_id = "UUtoNXlIegvxkvf5Ji8S57Ag"
                
# Get video IDs
LoreSoWhat_video_ids = get_video_ids(youtube, LoreSoWhat_id)

# Get video details
LoreSoWhat_video_df = get_video_details(youtube, LoreSoWhat_video_ids)
LoreSoWhat_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,VGqbzNq_EaI,Lore So What,SQL for Beginners in 60 seconds - pt.4,SQL in the most popular tool for data analysts...,"[sql for beginners 60 seconds, sql, programmin...",2023-08-08T13:00:20Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/VG...,224,18,0,2,PT58S,hd,false
1,m5KUevhWp6k,Lore So What,SQL for Beginners in 60 seconds - pt.3,SQL in the most popular tool for data analysts...,"[sql for beginners 60 seconds, sql, programmin...",2023-08-04T13:00:41Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/m5...,348,31,0,3,PT57S,hd,false
2,o9VpeM8vShE,Lore So What,Social Media Analytics in Tableau - Project Wa...,"If you are into data analytics, you might want...","[tableau project data analytics, data analytic...",2023-08-03T14:56:49Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/o9...,688,41,0,8,PT17M9S,hd,false
3,tfodS75d8n8,Lore So What,SQL for Beginners in 60 seconds - pt.2,SQL in the most popular tool for data analysts...,"[sql for beginners 60 seconds, sql, programmin...",2023-08-01T13:00:52Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/tf...,906,74,0,2,PT59S,hd,false
4,yTjYqGfMgBA,Lore So What,The ONE Tool to Grow on YouTube,Install TubeBuddy and use code LOREBUDDY20 for...,"[tool that made me 10000 subscribers, grow a y...",2023-07-31T12:12:10Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/yT...,213,9,0,2,PT57S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,7YMBgPgy_sY,Lore So What,Understanding ALL Data Roles,"At the start of my career, I clearly did not h...","[understanding data roles, data roles, data ro...",2022-11-02T18:55:23Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/7Y...,587,37,0,24,PT11M48S,hd,false
89,2Q1RMIXw8TM,Lore So What,The Data Scientist Role EXPLAINED,The Data Scientist role has been defined as th...,"[data scientist job explained, data scientist ...",2022-10-26T17:13:00Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/2Q...,511,36,0,26,PT12M58S,hd,false
90,NWSDYmJ9ZEY,Lore So What,THIS is Why I QUIT My Job,I recently made the tough decision to quit my ...,"[why i quit my consulting job, why i quitted m...",2022-10-19T12:31:35Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/NW...,825,38,0,28,PT12M,hd,false
91,92I-mZOC3os,Lore So What,This is Why YOU Should Move to Bali,Let me try to convince you why you should move...,"[why move to bali, why you should move to bali...",2022-10-12T11:06:58Z,22,en,{'default': {'url': 'https://i.ytimg.com/vi/92...,523,32,0,39,PT8M37S,hd,false


In [35]:
# LoreSoWhat_comment_df = get_comments_in_videos(youtube, LoreSoWhat_video_ids)
# LoreSoWhat_comment_df

In [36]:
# # file path to save the CSV file
# csv_file_path = 'LoreSoWhat_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# LoreSoWhat_video_df.to_csv(csv_file_path, index=False)

## **4.8- Thu Vu data analytics Channel**

In [37]:
ThuVu_id = "UUJQJAI7IjbLcpsjWdSzYz0Q"
                
# Get video IDs
ThuVu_video_ids = get_video_ids(youtube, ThuVu_id)

# Get video details
ThuVu_video_df = get_video_details(youtube, ThuVu_video_ids)
ThuVu_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,8sLq4P-QDL0,Thu Vu data analytics,I Tried ChatGPT Code Interpreter for Data Task...,I tell ChatGPT Code Interpreter to do my job 😅...,"[data analytics, data science, python, data, t...",2023-07-15T19:02:17Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/8s...,20057,987,0,35,PT58S,hd,false
1,3736Pd_hwFQ,Thu Vu data analytics,40 Productivity Tips that Make You More Pro in...,📩 Interested in hearing about my coming course...,"[data analytics, data science, python, data, t...",2023-07-08T09:45:30Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/37...,9609,534,0,56,PT1H16M9S,hd,false
2,PvnSSrnjLX8,Thu Vu data analytics,Self-taught vs. Guided Program: CareerFoundry'...,📊 Enroll with this link to get 10% off the Dat...,"[data analytics, data science, python, data, t...",2023-06-16T22:57:31Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Pv...,5806,187,0,32,PT16M32S,hd,false
3,S5U76LPu_bQ,Thu Vu data analytics,Is a Computer Science Degree WORTHLESS 🤯? (in ...,I've been thinking lately if it's still worth ...,"[data analytics, data science, python, data, t...",2023-06-08T16:10:11Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/S5...,12835,501,0,45,PT11M8S,hd,false
4,l2YU8QuXiTM,Thu Vu data analytics,How I Use Generative AI in Jupyter Notebook 🤩,⚙️ Jupyter AI 👉 https://jupyter-ai.readthedocs...,"[data analytics, data science, python, data, t...",2023-05-28T14:33:38Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/l2...,17312,752,0,63,PT12M40S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,5LWoJAh-kww,Thu Vu data analytics,Data Analyst Skill Stack // How I Became A Dat...,👩🏻‍💻 My laptop and iPad for doing DS/ study 👉 ...,0,2021-06-16T10:16:47Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/5L...,31644,1762,0,72,PT10M35S,hd,false
69,_RzoHVWKwq4,Thu Vu data analytics,Effective visual note-taking on iPad + Downloa...,Hi there! In this video I’m showing you how to...,"[Note-taking, Study, iPad Pro, Visual note-tak...",2021-06-09T06:58:21Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/_R...,8827,324,0,10,PT8M19S,hd,false
70,hWKLO7GtpiU,Thu Vu data analytics,Data scientist is NOT the only SEXY job // All...,Hi there! In this video I explain different da...,0,2021-06-01T22:51:42Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/hW...,8242,279,0,20,PT14M4S,hd,false
71,dBZqggW22rs,Thu Vu data analytics,"How I take notes on iPad Pro | Notion, Notes, ...",My note-taking system tour in Notion and tips ...,"[Notes, Study, Productivity, Notion, Procreate...",2021-05-16T22:47:12Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/dB...,9139,242,0,18,PT9M7S,hd,false


In [38]:
# ThuVu_comment_df = get_comments_in_videos(youtube, ThuVu_video_ids)
# ThuVu_comment_df

In [39]:
# # file path to save the CSV file
# csv_file_path = 'ThuVu_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# ThuVu_video_df.to_csv(csv_file_path, index=False)

## **4.9- Data With Mo Channel**

In [40]:
DataWithMo_id = "UUDybamfye5An6p-j1t2YMsg"
                
# Get video IDs
DataWithMo_video_ids = get_video_ids(youtube, DataWithMo_id)

# Get video details
DataWithMo_video_df = get_video_details(youtube, DataWithMo_video_ids)
DataWithMo_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,Fw5zziqSxk8,Data With Mo,Top Data Analyst Habits #5,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-07T14:19:47Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Fw...,1627,169,0,6,PT56S,hd,false
1,z-RFtw7QBUQ,Data With Mo,Top Data Analyst Habits #4,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-07T14:08:09Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/z-...,1011,88,0,1,PT58S,hd,false
2,YDbv2J-fQfI,Data With Mo,Top Data Analyst Habits #3.1 and #3.2,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-07T13:56:28Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/YD...,1119,91,0,3,PT50S,hd,false
3,HeyZy3W1Iw0,Data With Mo,Top Data Analyst Habits #2,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-07T13:43:42Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/He...,1208,114,0,3,PT50S,hd,false
4,XvYbHgBkg20,Data With Mo,Top Data Analyst Habits #1,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-07T13:26:49Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Xv...,1587,204,0,5,PT50S,hd,false
5,_8ia4G4RMHs,Data With Mo,How much Math do you need as a Data Analyst?,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-03T15:45:15Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/_8...,2490,214,0,10,PT52S,hd,false
6,ifV-3MEudcc,Data With Mo,3 Tips to begin your DATA ANALYST JOURNEY,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-03T15:33:31Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/if...,1847,224,0,4,PT54S,hd,false
7,_iCV6CdXscA,Data With Mo,Learn these TECHNICAL SKILLS to become a DATA...,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-03T13:44:51Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/_i...,2560,297,0,13,PT56S,hd,false
8,Zd9tgFBkOwg,Data With Mo,How did I get into Data & Analytics?,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-03T12:58:57Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/Zd...,1383,104,0,6,PT56S,hd,false
9,QOXFKi9ss8o,Data With Mo,How I'd Learn PYTHON For DATA ANALYSIS | If I ...,🛣️ Data Analyst Roadmap ➡︎ https://datawithmo....,0,2023-08-02T18:32:29Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/QO...,1322,156,0,8,PT45S,hd,false


In [41]:
# # file path to save the CSV file
# csv_file_path = 'DataWithMo_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# DataWithMo_video_df.to_csv(csv_file_path, index=False)

In [42]:
# DataWithMo_comment_df = get_comments_in_videos(youtube, DataWithMo_video_ids)
# DataWithMo_comment_df

## **4.10- Deena Gergis Channel**

In [43]:
DeenaGergis_id = "UU1FksTDit4ij9ogFAUC9xxg"
                
# Get video IDs
DeenaGergis_video_ids = get_video_ids(youtube, DeenaGergis_id)

# Get video details
DeenaGergis_video_df = get_video_details(youtube, DeenaGergis_video_ids)
DeenaGergis_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,MwKiJmpfpes,Deena Gergis,واحدة من أسخف مصادر البجز Shallow and Deep copies,الفيديو دة هيوريك واحدة من أهم مصادر البجز وإن...,"[pandas, data science, python, data analytics,...",2022-03-11T16:48:45Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/Mw...,2872,132,0,31,PT5M23S,hd,false
1,lYlhJnqNNIA,Deena Gergis,ازاي تتمكن من مهارة عرض البيانات بشكل مرئي - D...,تابع الحسابات دي عشان تشوف المحتوى المكمل\nFac...,"[python, plotly, dash, تعلم الالة, علم البيانا...",2021-09-03T16:47:54Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/lY...,7264,331,0,26,PT1M50S,hd,false
2,zQ5WqAz3myo,Deena Gergis,ApplAi interview: علم البيانات في المجالات الت...,ApplAi page: https://www.facebook.com/ApplAias...,"[Data Science, Artificial Intelligence, Machin...",2021-08-23T16:23:43Z,28,ar,{'default': {'url': 'https://i.ytimg.com/vi/zQ...,3672,183,0,40,PT1H22M34S,sd,true
3,28KvPGEpEhY,Deena Gergis,Live Q&A : أسئلتك لايف,,0,2021-08-07T07:38:31Z,28,0,{'default': {'url': 'https://i.ytimg.com/vi/28...,2073,115,0,8,PT1H31M13S,hd,false
4,YdBalS7mqTc,Deena Gergis,أهم 10 packages في علوم البيانات,Facebook: https://www.facebook.com/deena.gergi...,"[python, packages, pandas, pyspark, matplotlib...",2021-07-30T18:12:01Z,28,ar,{'default': {'url': 'https://i.ytimg.com/vi/Yd...,6706,470,0,65,PT11M14S,hd,false
5,CXeLJ72Siwk,Deena Gergis,Machine Learning يعني إيه,لو لسة عندك اسئلة سبهم في الكومنتس أو اتواصل م...,"[تعلم الالة, علم البيانات, البيانات الضخمة, ال...",2021-06-18T18:46:37Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/CX...,9097,440,0,54,PT7M41S,hd,false
6,ovr1MoIl6n0,Deena Gergis,Webcast - Exception Student Committee,لو لسة عندك اسئلة سبهم في الكومنتس أو اتواصل م...,"[Data Science, Artificial Intelligence, Machin...",2021-05-14T18:43:39Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/ov...,2279,113,0,24,PT1H32M5S,hd,true
7,gb7giYSa_8A,Deena Gergis,TecYouth Webinar : Data Science,,0,2021-05-07T22:59:23Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/gb...,1850,82,0,30,PT1H54M30S,hd,false
8,Z1u0oChSdI8,Deena Gergis,الذكاء الاصطناعي & Uber,لو لسة عندك اسئلة سبهم في الكومنتس أو اتواصل م...,"[Data Science, Artificial Intelligence, Machin...",2021-04-25T20:24:42Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/Z1...,3149,206,0,29,PT7M40S,hd,false
9,kjflN9m1d58,Deena Gergis,Live Q&A - اسئلتك لايف,,0,2021-04-10T08:35:43Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/kj...,2795,141,0,28,PT1H31M26S,hd,false


In [44]:
# DeenaGergis_comment_df = get_comments_in_videos(youtube, DeenaGergis_video_ids)
# DeenaGergis_comment_df

In [45]:
# # file path to save the CSV file
# csv_file_path = 'DeenaGergis_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# DeenaGergis_video_df.to_csv(csv_file_path, index=False)

## **4.11- techTFQ Channel**


In [46]:
techTFQ_id = "UUnz-ZXXER4jOvuED5trXfEA"
                
# Get video IDs
techTFQ_video_ids = get_video_ids(youtube, techTFQ_id)

# Get video details
techTFQ_video_df = get_video_details(youtube, techTFQ_video_ids)
techTFQ_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,D0-Qy7yk5TQ,techTFQ,SQL Data Analytics Project (PART 2) | Data Ana...,This is the SECOND part of the video series wh...,0,2023-08-06T13:00:08Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/D0...,6705,248,0,18,PT44M14S,hd,false
1,GHtX0QXfi6g,techTFQ,SQL Data Analytics Project (PART 1) | Data Ana...,This is the first part of the video series whe...,"[sql data analysis project, sql data analytics...",2023-08-04T13:00:11Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/GH...,20526,948,0,101,PT35M49S,hd,false
2,0QcEqFrhb_Y,techTFQ,Learn SQL from me | Live SQL Training | SQL Bo...,"In this video, I highlight the 3 reasons why m...","[sql, learn sql for beginners, learn sql, sql ...",2023-05-25T14:35:13Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/0Q...,21137,601,0,169,PT9M51S,hd,false
3,smztq8sRAhk,techTFQ,Learn Python here | Learn Python Programming here,"In this video, I will showcase a dedicated onl...","[learn python, python for beginners, python co...",2023-05-16T13:45:59Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/sm...,13815,259,0,23,PT6M30S,hd,false
4,7skZzocEU6c,techTFQ,Solving a Complex SQL Interview problem | Prac...,"In this video, let us solve a complex SQL inte...","[sql, learn sql, sql query, sql queries, sql p...",2023-03-21T13:00:12Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/7s...,33231,945,0,100,PT32M9S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1aybOgni7lI,techTFQ,How to install PostgreSQL on Mac OS | Install ...,This video is about how to install PostgreSQL ...,"[PostgreSQL, pgAdmin, Database, PostgreSQL Dat...",2020-11-16T02:28:09Z,27,en-IN,{'default': {'url': 'https://i.ytimg.com/vi/1a...,72311,689,0,98,PT20M51S,hd,false
87,j09EQ-xlh88,techTFQ,Learn What is Database | Types of Database | DBMS,"In this video, we learn everything we need to ...","[Database, DBMS, Relational Database, Non-Rela...",2020-08-30T00:38:24Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/j0...,215722,3821,0,123,PT12M11S,hd,false
88,7nzTDrio7vY,techTFQ,Do you need a Smartwatch,"In this video, I talk about the advantageous o...","[Do you need a smartwatch, Samsung Galaxy Watc...",2020-07-12T15:32:36Z,22,en-IN,{'default': {'url': 'https://i.ytimg.com/vi/7n...,11372,185,0,46,PT7M43S,hd,false
89,J-uCLHTIWZ4,techTFQ,MacBook Pro 13 2020 One Week Later Review,Macbook Pro 13 2020 review after one week. Qui...,0,2020-06-29T15:03:19Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/J-...,1389,71,0,20,PT9M24S,hd,false


In [47]:
# # file path to save the CSV file
# csv_file_path = 'techTFQ_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# techTFQ_video_df.to_csv(csv_file_path, index=False)

In [48]:
# techTFQ_comment_df = get_comments_in_videos(youtube, techTFQ_video_ids)
# techTFQ_comment_df

## **4.12- Learn with Lukas Channel**

In [49]:
LearnwithLukas_id = "UUw_LFe2pS8x3NyipGNJgeEA"
                
# Get video IDs
LearnwithLukas_video_ids = get_video_ids(youtube, LearnwithLukas_id)

# Get video details
LearnwithLukas_video_df = get_video_details(youtube, LearnwithLukas_video_ids)
LearnwithLukas_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,ISnKvYw-KVE,Learn with Lukas,"$100 Data Analytics Certificate vs $100,000 De...",Let's compare a degree vs a certificate for as...,0,2023-08-07T07:11:59Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/IS...,1274,65,0,19,PT8M59S,hd,true
1,nX7GCf2bsHA,Learn with Lukas,BEST Course to Become a Data Analyst?!? The SQ...,Learning SQL does not have to involve sufferin...,"[sql for beginners, sql tutorial, sql, learn s...",2023-08-03T22:00:26Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/nX...,2403,115,0,21,PT5M49S,hd,false
2,nnIy5SYEr40,Learn with Lukas,Become a Data Engineer with NO degree? IBM Dat...,Let's see if this is the best way to become a ...,"[ibm data science, ibm data engineering, cours...",2023-07-22T14:22:56Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/nn...,2435,122,0,18,PT6M14S,hd,false
3,tFK6q8__vsw,Learn with Lukas,"Data Analysts, Do You ACTUALLY Know Excel?","Excel is an often forgotten, but incredibly us...","[microsoft excel, excel tips and tricks, excel...",2023-07-09T19:12:23Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/tF...,4957,232,0,15,PT3M41S,hd,true
4,LBSN34puYcQ,Learn with Lukas,50 Data Analyst Concepts You Must Know,Let's cover 50 data analytics concepts in just...,"[data analyst, data analytics, data analyst jo...",2023-07-08T07:52:35Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/LB...,3721,279,0,29,PT6M2S,hd,true
5,nLkunQXKceA,Learn with Lukas,5 Best Reasons Not To Become a Data Analyst,"While becoming a data analyst is great, it's n...","[data analytics, data analyst job, data analys...",2023-07-02T18:00:10Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/nL...,27848,475,0,34,PT4M16S,hd,true
6,apEabfi2o1M,Learn with Lukas,Data Analyst vs Data Engineer,"The cloud storage I use (free 3 MONTH trial, c...","[Data Analyst, How to become a data analyst, D...",2023-06-30T13:48:49Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/ap...,3213,162,0,12,PT7M19S,hd,false
7,tb7UFCKM-J4,Learn with Lukas,AI Could Replace Data Analysts Soon,"I really didn't want to make this video, but I...","[data analytics, data science, data analyst, d...",2023-06-17T08:48:40Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/tb...,8097,291,0,36,PT4M47S,hd,true
8,gzInkWJHx18,Learn with Lukas,Fastest Way To Scrape Data As a Data Analyst,Web scraping is an essential data skill. Let's...,"[web scraping, web scraping tutorial, data scr...",2023-06-01T22:00:21Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/gz...,4772,203,0,5,PT7M18S,hd,true
9,HRQ0vUodSuE,Learn with Lukas,Data Analyst Certificates Are Dead,Data analyst certificates are not what they us...,"[data analytics, data analyst, data analytics ...",2023-05-27T09:36:01Z,26,en,{'default': {'url': 'https://i.ytimg.com/vi/HR...,232457,9541,0,269,PT5M38S,hd,true


In [50]:
# # file path to save the CSV file
# csv_file_path = 'LearnwithLukas_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# LearnwithLukas_video_df.to_csv(csv_file_path, index=False)

In [51]:
# LearnwithLukas_comment_df = get_comments_in_videos(youtube, LearnwithLukas_video_ids)
# LearnwithLukas_comment_df

## **4.13- Alex The Analyst Channel**

In [52]:
AlexTheAnalyst_id = "UU7cs8q-gJRlGwj4A8OmCmXg"
                
# Get video IDs
AlexTheAnalyst_video_ids = get_video_ids(youtube, AlexTheAnalyst_id)

# Get video details
AlexTheAnalyst_video_df = get_video_details(youtube, AlexTheAnalyst_video_ids)
AlexTheAnalyst_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,nufF-juSZxU,Alex The Analyst,LINK: https://www.youtube.com/playlist?list=PL...,,"[Data Analyst, Data Analyst job, Data Analyst ...",2023-08-09T13:11:45Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/nu...,2525,306,0,24,PT37S,hd,false
1,9NB5u17qiDY,Alex The Analyst,Regular Expression Character Classes in Python,In this lesson we are going to look at Charact...,"[Data Analyst, Data Analyst job, Data Analyst ...",2023-08-08T12:00:05Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/9N...,3763,83,0,38,PT10M38S,hd,false
2,YDvJqgBv0Uc,Alex The Analyst,#Recruiters #dataanalysis #data,,0,2023-08-03T11:56:30Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/YD...,6325,417,0,20,PT58S,hd,false
3,j6A28L6Tmxw,Alex The Analyst,Regular Expression Metacharacters in Python,In this lesson we are going to look at Metacha...,"[Data Analyst, Data Analyst job, Data Analyst ...",2023-08-01T12:00:26Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/j6...,3150,87,0,23,PT20M3S,hd,false
4,W_s4c1vLGXM,Alex The Analyst,How much do Data Analysts make? #dataanalyst #...,,0,2023-07-31T13:07:06Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/W_...,8773,363,0,30,PT46S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,4rfr6A3lO-Y,Alex The Analyst,Data Analyst Resume | Reviewing My Resume! | F...,Data Analyst Resume | Reviewing My Resume! | F...,"[Data Analyst, How to become a data analyst, D...",2020-01-30T14:07:55Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/4r...,63385,1538,0,64,PT7M33S,hd,false
226,OTq2NRy_AGs,Alex The Analyst,Working at a Big Company Vs Small Company | To...,Working at a Big Company Vs Small Company | To...,"[Data Analyst, How to become a Data Analyst, B...",2020-01-25T16:38:39Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/OT...,13946,383,0,20,PT5M50S,hd,false
227,ya28cb3zFGE,Alex The Analyst,Data Analyst Salary | 100k with No Experience,Data Analyst Salary | 100k with No Experience ...,"[Data Analyst Salary, Data analyst with no exp...",2020-01-23T03:16:09Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/ya...,60189,2098,0,225,PT5M3S,hd,false
228,Hsi2BG0SOiQ,Alex The Analyst,Truth About Big Companies | Told by a Fortune ...,Truth About Big Companies // There are a ton o...,"[Working at a big company, Big company data an...",2020-01-21T03:52:15Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Hs...,7596,289,0,18,PT5M45S,hd,false


In [53]:
# # file path to save the CSV file
# csv_file_path = 'AlexTheAnalyst_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# AlexTheAnalyst_video_df.to_csv(csv_file_path, index=False)

In [54]:
# AlexTheAnalyst_comment_df = get_comments_in_videos(youtube, AlexTheAnalyst_video_ids)
# AlexTheAnalyst_comment_df

## **4.14- Codebasics Channel**

In [55]:
codebasics_id = "UUh9nVJoWXmFb7sLApWGcLPQ"
                
# Get video IDs
codebasics_video_ids = get_video_ids(youtube, codebasics_id)

# Get video details
codebasics_video_df = get_video_details(youtube, codebasics_video_ids)
codebasics_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,bd3KtLYRpB4,codebasics,Data Analyst Job-Ready Checklist: Know Where Y...,The worst thing that can happen to data analys...,"[yt:cc=on, data analyst jobs, data analyst, da...",2023-08-04T14:00:11Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/bd...,21843,1032,0,104,PT11M1S,hd,false
1,uSLQKYsEYMo,codebasics,Data Analyst at FORD after 7 years in Mechanic...,Raghavan transitioned to a fine data analyst a...,"[yt:cc=on, career transition, mechanical engin...",2023-08-01T13:30:12Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/uS...,11525,302,0,30,PT39M32S,hd,false
2,Rm0xH2Vpfi0,codebasics,SQL Tutorial For Beginners | MySQL Tutorial,Beginners SQL tutorial with code and exercises...,"[yt:cc=on, sql, sql tutorial, sql tutorial for...",2023-07-29T14:00:26Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/Rm...,12094,402,0,60,PT1H26M10S,hd,false
3,npgbI8KYvN8,codebasics,Business Math & Statistics Using Excel For Dat...,Learn this 20 % business math and statistics t...,"[yt:cc=on, excel tutorial for beginners, data ...",2023-07-21T13:30:09Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/np...,13680,427,0,20,PT1H24M49S,hd,false
4,tku5zP1VzXA,codebasics,How to use ChatGPT on your data - End to End P...,"In this beginners' video, we will go over what...","[yt:cc=on, openai function calling, openai fun...",2023-07-12T13:30:12Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/tk...,16565,449,0,38,PT42M47S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,pP91kLR5cnE,codebasics,Python Tutorial - 5. Lists,This python video answers about lists. It give...,"[python, python tutorial, python for beginners...",2015-12-20T23:01:51Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/pP...,34313,316,0,55,PT12M4S,hd,false
609,PsoKq0p5meU,codebasics,Python Tutorial - 4. Strings,"In this video of the python tutorial, we will ...","[python, python tutorial, python for beginners...",2015-12-20T22:34:19Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Ps...,28503,316,0,28,PT9M27S,hd,false
610,YJKEqaaSGDs,codebasics,Python Tutorial - 3. Numbers,This python tutorial gives you an understandin...,"[python, python tutorial, python for beginners...",2015-12-20T02:54:56Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/YJ...,32213,318,0,31,PT10M,hd,false
611,BAoV5_qVKco,codebasics,Python Tutorial - 2. Variables,"In this Python tutorial, we will learn about v...","[python, python tutorial, python for beginners...",2015-12-20T02:11:10Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/BA...,42379,333,0,24,PT4M45S,hd,false


In [56]:
# codebasics_comment_df = get_comments_in_videos(youtube, codebasics_video_ids)
# codebasics_comment_df

In [57]:
# # file path to save the CSV file
# csv_file_path = 'codebasics_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# codebasics_video_df.to_csv(csv_file_path, index=False)

## **4.15- Mustafa Othman Channel**

In [58]:
MustafaOthman_id = "UUITfscBBm40O9MgtqiQC00w"
                
# Get video IDs
MustafaOthman_video_ids = get_video_ids(youtube, MustafaOthman_id)

# Get video details
MustafaOthman_video_df = get_video_details(youtube, MustafaOthman_video_ids)
MustafaOthman_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,CYPdmobP4os,Mustafa Othman,تطبيقات علم البيانات: المحاضرة الثامنة | Data ...,لأي تفاصيل عن دوراتنا ولأي تساؤلات .. نتشرف بت...,0,2023-07-26T04:21:21Z,24,ar,{'default': {'url': 'https://i.ytimg.com/vi/CY...,846,34,0,5,PT42M,hd,false
1,QnX9DSpVMhc,Mustafa Othman,تطبيقات علم البيانات: المحاضرة السابعة | Data ...,لأي تفاصيل عن دوراتنا ولأي تساؤلات .. نتشرف بت...,0,2023-07-25T04:27:46Z,24,ar,{'default': {'url': 'https://i.ytimg.com/vi/Qn...,940,48,0,4,PT50M25S,hd,false
2,BK4R2JJ1EKc,Mustafa Othman,عقلية محلل البيانات: خدلك غطس في الداتا,موقع من المواقع المتميزة جداً في مجال الذكاء ا...,0,2023-06-20T06:56:12Z,24,ar,{'default': {'url': 'https://i.ytimg.com/vi/BK...,3234,132,0,14,PT57M48S,hd,false
3,vP2eEZIk47E,Mustafa Othman,برنامج التوجيه والإرشاد: علم البيانات والتسويق...,للتسجيل: https://bit.ly/tq-mentor\nللتفاصيل: h...,0,2023-05-26T12:59:28Z,27,ar,{'default': {'url': 'https://i.ytimg.com/vi/vP...,2025,133,0,10,PT1M1S,hd,false
4,Hf_4mNwvZeM,Mustafa Othman,تطبيقات علم البيانات: المحاضرة السادسة | Data ...,لأي تفاصيل عن دوراتنا ولأي تساؤلات .. نتشرف بت...,0,2023-04-06T12:23:31Z,24,ar,{'default': {'url': 'https://i.ytimg.com/vi/Hf...,1783,66,0,10,PT1H8M16S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,iJS4cngw8Jw,Mustafa Othman,كورس التسويق الالكتروني: (6) تهيئة / تسويق محر...,تقدر تشوف كل كورسات دبلومة التسويق الرقمي المت...,"[Search Engine Optimization (Interest), Web De...",2015-04-10T18:39:10Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/iJ...,3666,65,0,1,PT1H33M46S,sd,false
356,RSdlTdOVxyc,Mustafa Othman,كورس التسويق الالكتروني: (2) إزاي تعمل الخطة ا...,تقدر تشوف كل كورسات دبلومة التسويق الرقمي المت...,"[Marketing (Interest), Marketing Plan, Digital...",2015-04-10T16:45:37Z,22,ar,{'default': {'url': 'https://i.ytimg.com/vi/RS...,11784,142,0,6,PT20M1S,sd,false
357,STIHwVeY7-w,Mustafa Othman,كورس التسويق الالكتروني: (5) يعني إيه تسويق شب...,تقدر تشوف كل كورسات دبلومة التسويق الرقمي المت...,"[Marketing (Interest), Social Media (Website C...",2015-04-09T18:07:24Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/ST...,3768,39,0,2,PT26M,sd,false
358,E8yoq6jB6Jo,Mustafa Othman,كورس التسويق الالكتروني: (4) يعني إيه تسويق ال...,تقدر تشوف كل كورسات دبلومة التسويق الرقمي المت...,"[Marketing (Interest), Digital Marketing (Indu...",2015-04-09T15:26:13Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/E8...,5779,101,0,6,PT31M48S,sd,false


In [59]:
# MustafaOthman_comment_df = get_comments_in_videos(youtube, MustafaOthman_video_ids)
# MustafaOthman_comment_df

In [60]:
# # file path to save the CSV file
# csv_file_path = 'MustafaOthman_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# MustafaOthman_video_df.to_csv(csv_file_path, index=False)

## **4.16- Justin Shin Channel**

In [61]:
JustinShin_id = "UUe4c5YQfFsCQVMsOnJOiTIw"
                
# Get video IDs
JustinShin_video_ids = get_video_ids(youtube, JustinShin_id)

# Get video details
JustinShin_video_df = get_video_details(youtube, JustinShin_video_ids)
JustinShin_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,QKUg87dahME,Justin Shin,How I'm learning Data Science as a Data Analyst,"In this video, I'll be going over how I'm lear...","[Datavisualization, Tech, Excel, Programming, ...",2023-07-31T12:00:20Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/QK...,11597,377,0,26,PT8M10S,hd,false
1,abhkbsttjoc,Justin Shin,Why you should go to Hawaii,I went to Hawaii with my wife for our honeymoo...,"[Hawaii, Cinematography, Honeymoon]",2023-07-10T23:30:01Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/ab...,555,32,0,7,PT3M51S,hd,false
2,nr8ZH0sd3wg,Justin Shin,Why growing up is painful,Sharing some thoughts about growing up and lea...,"[Cinematography, growingup, comingofage, life]",2023-06-13T22:45:01Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/nr...,1073,88,0,9,PT5M1S,hd,false
3,xmgtUg_6Bcs,Justin Shin,The most dangerous hike in America,I went on my bachelor trip to a canyon out in ...,"[cinematography, hiking, bachelorparty, vlog]",2023-05-08T23:00:22Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/xm...,730,37,0,36,PT9M14S,hd,false
4,6EefZkGQPDo,Justin Shin,I Have Anxiety,Sharing about my struggles with Anxiety. \n\n\...,"[Anxiety, MentalHealth, Cinematography]",2023-04-11T23:00:03Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/6E...,1309,64,0,19,PT6M29S,hd,false
5,QM3ll8wurFU,Justin Shin,Data Scientist vs Data Analyst | A Deep Dive,To try everything Brilliant has to offer—free—...,"[Datavisualization, Tech, Excel, Programming, ...",2023-03-16T12:00:27Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/QM...,6765,458,0,11,PT7M29S,hd,false
6,XxWNdJxX7Ko,Justin Shin,I found the Best View in LA,I went to LA this past January for a week with...,"[date, minigolf, cooking, cinematography]",2023-02-14T00:00:23Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Xx...,992,149,0,15,PT7M55S,hd,false
7,9q6pgF4KMxY,Justin Shin,A Day in the life of a Data Analyst in Los Ang...,In this video I'll be sharing my day as a data...,"[dataanalytics, dataanalyst, dayinthelife, wor...",2023-01-31T15:00:06Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/9q...,22524,611,0,52,PT6M14S,hd,false
8,RR_78zP7XV4,Justin Shin,Why This Year Changed My Life,What a year it's been. \n\n\n\n\n#2022 #fx3,"[2022, Recap, Fx3]",2023-01-01T03:15:03Z,1,0,{'default': {'url': 'https://i.ytimg.com/vi/RR...,2408,183,0,32,PT3M49S,hd,false
9,0BRDh49t8mo,Justin Shin,A Day in the life of a Data Analyst in New Yor...,In this video I'll be sharing my day as a data...,"[dataanalytics, dataanalyst, dayinthelife, wor...",2022-12-28T15:00:00Z,22,0,{'default': {'url': 'https://i.ytimg.com/vi/0B...,199108,5839,0,248,PT9M45S,hd,false


In [62]:
# # file path to save the CSV file
# csv_file_path = 'JustinShin_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# JustinShin_video_df.to_csv(csv_file_path, index=False)

In [63]:
# JustinShin_comment_df = get_comments_in_videos(youtube, JustinShin_video_ids)
# JustinShin_comment_df

## **4.17- Ken Jee Channel**

In [64]:
KenJee_id = "UUiT9RITQ9PW6BhXK0y2jaeg"
                
# Get video IDs
KenJee_video_ids = get_video_ids(youtube, KenJee_id)

# Get video details
KenJee_video_df = get_video_details(youtube, KenJee_video_ids)
KenJee_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,6BrYKdqq4uA,Ken Jee,What's in My Data Science Travel Bag? (50+ Fli...,#DataScience #kenjee \n\nI travel a LOT for w...,"[Data Science, Ken Jee, data science journey, ...",2023-08-07T17:57:05Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/6B...,2490,84,0,38,PT10M12S,hd,false
1,1dAvbIMrLos,Ken Jee,7 Enticing Jobs AI Will Create,#DataScience #KenJee #AI\nIn this video I walk...,"[Ken Jee, artificial intelligence, chat gpt, f...",2023-07-24T12:00:01Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/1d...,3661,168,0,33,PT8M24S,hd,false
2,lqYxmYMkfK8,Ken Jee,The ChatGPT Code Interpreter is OVERRATED,"#DataScience #KenJee #ai \n\nIn this video, I ...","[Data Science, Ken Jee, Machine Learning, data...",2023-07-22T09:42:29Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/lq...,3490,127,0,29,PT5M8S,hd,false
3,-JTU734oMfk,Ken Jee,Exciting Announcement!,#DataScience #KenJee \nBig news! I know I have...,"[Data Science, Ken Jee, Machine Learning, data...",2023-06-23T17:28:36Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/-J...,6262,284,0,48,PT4M50S,hd,false
4,HVpbmu7rFYE,Ken Jee,AI: A Customer Service Revolution?,#DataScience #KenJee #ai \n\nThis past week I ...,"[Data Science, Ken Jee, Machine Learning, data...",2023-05-26T12:00:21Z,22,en-US,{'default': {'url': 'https://i.ytimg.com/vi/HV...,3416,114,0,35,PT9M44S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,irjTWNV0eAY,Ken Jee,How to Simulate NBA Games in Python,In this video I show you how to simulate NBA G...,"[Simulation, NBA, NBA Simulation, Basketball, ...",2018-11-19T23:31:49Z,17,en,{'default': {'url': 'https://i.ytimg.com/vi/ir...,17590,390,0,54,PT9M16S,hd,false
276,RRSRKf9eQxc,Ken Jee,Should You Get A Masters in Data Science?,Is a Masters in data science degree worth it? ...,"[Data, Data Science, Masters in Data Science, ...",2018-11-14T01:39:33Z,27,0,{'default': {'url': 'https://i.ytimg.com/vi/RR...,20497,306,0,135,PT5M37S,hd,false
277,IFceyuL6GZY,Ken Jee,How I Became A Data Scientist From a Business ...,In this video I provide recommendations on how...,"[Data Science, Machine Learning, kaggle, ken j...",2018-11-12T00:26:08Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/IF...,6416,197,0,23,PT10M19S,hd,false
278,Y_SMU701qlA,Ken Jee,Predicting Season Long NBA Wins Using Multiple...,This is my Final Project Presentation for CSC ...,"[Regression, Statistics, basketball analytics,...",2018-07-10T16:39:52Z,17,en,{'default': {'url': 'https://i.ytimg.com/vi/Y_...,8624,193,0,17,PT8M59S,hd,false


In [65]:
# # file path to save the CSV file
# csv_file_path = 'KenJee_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# KenJee_video_df.to_csv(csv_file_path, index=False)

In [66]:
# KenJee_comment_df = get_comments_in_videos(youtube, KenJee_video_ids)
# KenJee_comment_df

## **4.18- Youssef Hosni Channel**

In [67]:
YoussefHosni_id = "UUeEcSgRzYFuVt-2Yk1ULdhQ"
                
# Get video IDs
YoussefHosni_video_ids = get_video_ids(youtube, YoussefHosni_id)

# Get video details
YoussefHosni_video_df = get_video_details(youtube, YoussefHosni_video_ids)
YoussefHosni_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,OJcjncuHn60,Youssef Hosni,إلى البيانات و ما بعدها (10) | م. أحمد عمر عيس...,إلى البيانات و ما بعدها (10) | م. أحمد عمر عيس...,0,2023-08-10T17:00:20Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/OJ...,92,10,0,2,PT58M,sd,false
1,oz102I6Woek,Youssef Hosni,نحو قراءة أكثر فاعلية للكتب التقنية | إضاءات ع...,نحو قراءة أكثر فاعلية للكتب التقنية | إضاءات ع...,0,2023-08-07T17:00:34Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/oz...,136,17,0,8,PT11M27S,hd,false
2,CMMcIFRwTUk,Youssef Hosni,إلى البيانات و ما بعدها (9) | د. محمد رجب | A*...,إلى البيانات و ما بعدها (9) | د. محمد رجب | A*...,0,2023-08-03T17:00:08Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/CM...,218,14,0,2,PT1H53M17S,hd,false
3,WG9ahEZOzog,Youssef Hosni,Crack the Data Interview | 5 -What to Expect i...,Crack the Data Interview | 5 -What to Expect i...,0,2023-07-29T15:37:18Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/WG...,202,17,0,1,PT15M40S,hd,false
4,G2Ym8RMvNHg,Youssef Hosni,بودكاست إلى البيانات و ما بعدها (8) كل ما تود ...,بودكاست إلى البيانات و ما بعدها | كل ما تود مع...,0,2023-07-27T17:00:10Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/G2...,445,27,0,5,PT1H33M16S,hd,false
5,yB7_teLsPMQ,Youssef Hosni,Add to Your Library | 1- Ten End-to-End Projec...,Add to Your Library | 1- Ten End-to-End Projec...,0,2023-07-23T17:00:43Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/yB...,785,54,0,2,PT7M49S,hd,false
6,V1dOMnrtkOM,Youssef Hosni,Practical Machine Learning | 1- Introduction |...,Practical Machine Learning | 1- Introduction |...,0,2023-07-16T17:00:24Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/V1...,551,41,0,9,PT6M7S,hd,false
7,nESv7LsYpDs,Youssef Hosni,الحلقة 16 | سورة الجن | وَأَنَّهُمۡ ظَنُّواْ...,وقفة ايمانيه مع قوله تعالى وَأَنَّهُمۡ ظَنُّوا...,0,2023-04-14T19:01:27Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/nE...,144,9,0,0,PT3M19S,hd,false
8,t4qg-AFtYYA,Youssef Hosni,الحلقة 15 | سورة الجن | يَهْدِي إِلَى الرُّشْدِ,,0,2023-04-10T17:38:51Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/t4...,77,4,0,0,PT4M30S,hd,false
9,f3jW9HoINns,Youssef Hosni,الحلقة 14 | سورة نوح | فَقُلْتُ اسْتَغْفِرُو...,,0,2023-04-08T17:36:06Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/f3...,48,3,0,1,PT5M13S,hd,false


In [68]:
# YoussefHosni_comment_df = get_comments_in_videos(youtube, YoussefHosni_video_ids)
# YoussefHosni_comment_df

In [69]:
# # file path to save the CSV file
# csv_file_path = 'YoussefHosni_channel_stats.csv'

# # Save the channel_stats DataFrame to a CSV file
# YoussefHosni_video_df.to_csv(csv_file_path, index=False)

## **4.19- Krish Naik Channel**

In [70]:
KrishNaik_id = "UUNU_lfiiWBdtULKOw6X0Dig"
                
# Get video IDs
KrishNaik_video_ids = get_video_ids(youtube, KrishNaik_id)

# Get video details
KrishNaik_video_df = get_video_details(youtube, KrishNaik_video_ids)
KrishNaik_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,RnzC2ULvp5I,Krish Naik,Tanisha Transition Story To Data Analytics Ind...,"Hello Guys,\nToday we will hear about the amaz...","[yt:cc=on, data analyst jobs, how to crack dat...",2023-08-03T11:52:28Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/Rn...,23373,364,0,53,PT19M38S,hd,false
1,PxCCzJndruQ,Krish Naik,Tavishi Transition Story To Data Science With ...,"Hello Guys,\nToday we will hear about the amaz...","[yt:cc=on, Data science transition story, how ...",2023-07-30T13:57:37Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/Px...,53229,1088,0,122,PT25M25S,hd,false
2,pxk1Fr33-L4,Krish Naik,End To End MLOPS Data Science Project Implemen...,In this project we will be implementing end to...,"[yt:cc=on, data science project implementation...",2023-07-28T12:08:18Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/px...,15054,547,0,50,PT2H32M35S,hd,false
3,mHQPzVse2oA,Krish Naik,Data Versioning Hands On Tutorials With DVC Fo...,DVC is a tool for data science that takes adva...,"[yt:cc=on, dvc tutorials, data versioning, mac...",2023-07-25T11:35:59Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/mH...,7529,192,0,18,PT26M27S,hd,false
4,qdcHHrsXA48,Krish Naik,The Power Of MLFLOW And Dagshub-Open Source Da...,MLflow is an open source platform to manage th...,"[yt:cc=on, mlflow tutorials, dagshub tutorials...",2023-07-23T05:42:25Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/qd...,12333,384,0,23,PT41M14S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,MTiaCUh1420,Krish Naik,Important libraries used in python Data Scienc...,Important libraries used in python Data Scienc...,"[Machine Learning, Artificial Intelligence, Da...",2017-11-26T07:48:27Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/MT...,20326,177,0,15,PT8M31S,sd,false
1628,DeT8mji0Jos,Krish Naik,Anaconda installation with Packages- Machine L...,Detailed explanation of anaconda python instal...,"[Machine Learning, Artificial Intelligence, Da...",2017-11-26T06:16:34Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/De...,75598,344,0,38,PT5M18S,sd,false
1629,HrHJUc26YxI,Krish Naik,What is Supervised Machine Learning- Machine L...,Detailed Explanation of Supervised Machine Lea...,"[Machine Learning basics, Artificial Intellige...",2017-11-26T04:50:08Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/Hr...,34595,397,0,16,PT11M42S,sd,false
1630,EqRsD3gqeCo,Krish Naik,What is Machine Learning in Data Science- Mach...,"Detailed explanation of Machine Learning ,type...","[Machine learning basics and types, Data Scien...",2017-11-25T12:27:20Z,27,en,{'default': {'url': 'https://i.ytimg.com/vi/Eq...,138419,858,0,48,PT10M,hd,false


## **4.20- How to Power BI**

In [72]:
HowtoPowerBI_id = "UUcfngi7_ASuo5jdWX0bNauQ"
                
# Get video IDs
HowtoPowerBI_video_ids = get_video_ids(youtube, HowtoPowerBI_id)

# Get video details
HowtoPowerBI_video_df = get_video_details(youtube, HowtoPowerBI_video_ids)
HowtoPowerBI_video_df

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,1t7mXIQIMe4,How to Power BI,DAX Basics: Filter CONTEXT & Calculate | Powe...,🚀 DAX is overwhelming and understanding these ...,"[dax power bi, dax, power bi dax, powerbi dax,...",2023-08-09T12:45:00Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/1t...,3466,206,0,3,PT1M,hd,false
1,uq6pjrxz0VU,How to Power BI,🔥Icon SLICER BUTTONS | Power BI Trick #PowerBI...,Here something you may not know- how to make s...,"[how to power bi, report in power bi, power bi...",2023-08-03T09:15:03Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/uq...,4833,280,0,0,PT37S,hd,false
2,vyU1Y8_FFdY,How to Power BI,Create a UNIQUE Slicer | NEW CARD Visual + SLI...,Here is an idea of how you can combine New Car...,"[how to power bi, report in power bi, power bi...",2023-07-28T09:45:05Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/vy...,9175,410,0,3,PT1M,hd,false
3,CN7WpgdCXNI,How to Power BI,Quick Guide: FIELDS PARAMETERS in Power BI #P...,Here is a quick implementation of fields param...,"[how to power bi, report in power bi, power bi...",2023-07-26T09:00:06Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/CN...,5598,394,0,4,PT1M,hd,false
4,bFBKm7uSw4U,How to Power BI,QUICK Power BI REPORT Background DESIGN #Powe...,Here is the best tip for ultra quick report de...,"[power bi visualization, power bi report, powe...",2023-07-24T14:30:11Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/bF...,6635,306,0,3,PT1M,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,O7aL5v1r6Q4,How to Power BI,DRILLDOWN and DRILLTHROUGH in Power BI - Taki...,Learn how to use drilldown and drillthrough in...,"[power bi drill through, power bi drill down, ...",2020-06-18T11:00:07Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/O7...,87736,1607,0,108,PT12M55S,hd,false
292,_HTF7Ph7Eqc,How to Power BI,5 WAYS to use BOOKMARKS in Power BI,Learn how to use bookmarks in Power BI. \nBook...,"[power bi bookmarks, bookmarks in power bi, ho...",2020-06-15T11:00:22Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/_H...,68700,1513,0,99,PT9M54S,hd,false
293,mOaP4jvOd8k,How to Power BI,Merging Data with FUZZY MATCH in Power BI,Merging data with fuzzy match in Power BI.\n\n...,"[fuzzy match, fuzzy merge, fuzzy match power b...",2020-05-12T14:19:13Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/mO...,4828,67,0,5,PT8M52S,hd,false
294,x64KFTbk4Yc,How to Power BI,CONDITIONAL FORMATTING in Power BI I All you n...,Bring some colour to your life with conditiona...,"[conditional formatting, formatting power bi, ...",2020-05-03T14:17:42Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/x6...,6786,136,0,8,PT22M21S,hd,false


## **5- Saving Data**

In [73]:
youtubedataframes = [SundasKhalid_video_df, Annamonova_video_df, LukeBarousse_video_df, edrea_video_df, RaneshGuruparan_video_df, 
            MohamedAlAssaal_video_df, LoreSoWhat_video_df, ThuVu_video_df, DataWithMo_video_df, DeenaGergis_video_df, techTFQ_video_df, 
            LearnwithLukas_video_df, AlexTheAnalyst_video_df, codebasics_video_df, MustafaOthman_video_df, JustinShin_video_df, KenJee_video_df,
            YoussefHosni_video_df, KrishNaik_video_df, HowtoPowerBI_video_df]

# Concatenate the dataframes
youtube_data = pd.concat(youtubedataframes, ignore_index=True)
youtube_data

,video_id,channelTitle,title,description,tags,publishedAt,categoryId,defaultAudioLanguage,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption
0,l14K2EnD548,Sundas Khalid,AI Will Replace Tech Jobs: From ex-FAANG Softw...,50 job interview questions & answers 👉🏼 https:...,"[data science, self-taugh data scientist, tech...",2023-08-07T14:10:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/l1...,4324,149,0,30,PT30M46S,hd,true
1,7ssLi7Ll0I0,Sundas Khalid,How Much Money I Made as Data Engineer? (3 yea...,Resume & Cover Letter template (free) 👉🏼 https...,"[data science, data scientist, self-taugh data...",2023-07-28T14:10:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/7s...,16491,296,0,30,PT6M14S,hd,true
2,xr68cbOxvBs,Sundas Khalid,How to learn Python FAST with ChatGPT and Bard?,Try Quadratic for FREE 👉🏼 https://QuadraticHQ....,"[data science, data scientist, self-taugh data...",2023-07-10T14:10:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/xr...,334755,3954,0,105,PT12M6S,hd,true
3,mLP4kdk3DoI,Sundas Khalid,Will AI Replace Data Scientists?,Excel graphs template (free) 👉🏼 https://clickh...,"[data science, data scientist, self-taugh data...",2023-06-29T14:12:00Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/mL...,18049,551,0,70,PT8M37S,hd,false
4,znouY2A61WI,Sundas Khalid,How to code Python FAST for Data Analysis: Bar...,Click to read full AI Trend Report (FREE) 👉🏼 h...,"[data science, data scientist, self-taugh data...",2023-06-16T14:10:01Z,28,en,{'default': {'url': 'https://i.ytimg.com/vi/zn...,23034,471,0,39,PT9M24S,hd,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4665,O7aL5v1r6Q4,How to Power BI,DRILLDOWN and DRILLTHROUGH in Power BI - Taki...,Learn how to use drilldown and drillthrough in...,"[power bi drill through, power bi drill down, ...",2020-06-18T11:00:07Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/O7...,87736,1607,0,108,PT12M55S,hd,false
4666,_HTF7Ph7Eqc,How to Power BI,5 WAYS to use BOOKMARKS in Power BI,Learn how to use bookmarks in Power BI. \nBook...,"[power bi bookmarks, bookmarks in power bi, ho...",2020-06-15T11:00:22Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/_H...,68700,1513,0,99,PT9M54S,hd,false
4667,mOaP4jvOd8k,How to Power BI,Merging Data with FUZZY MATCH in Power BI,Merging data with fuzzy match in Power BI.\n\n...,"[fuzzy match, fuzzy merge, fuzzy match power b...",2020-05-12T14:19:13Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/mO...,4828,67,0,5,PT8M52S,hd,false
4668,x64KFTbk4Yc,How to Power BI,CONDITIONAL FORMATTING in Power BI I All you n...,Bring some colour to your life with conditiona...,"[conditional formatting, formatting power bi, ...",2020-05-03T14:17:42Z,28,en-US,{'default': {'url': 'https://i.ytimg.com/vi/x6...,6786,136,0,8,PT22M21S,hd,false


In [74]:
# Save the channel_stats DataFrame to a CSV file
channel_stats.to_csv('/Users/godzilla/Desktop/Selected/Project/scraped_data/channel_stats.csv', index=False)

In [75]:
# Save the channel_stats DataFrame to a CSV file
youtube_data.to_excel('/Users/godzilla/Desktop/Selected/Project/scraped_data/channel_data.xlsx', index=False)

/Users/godzilla/opt/anaconda3/lib/python3.9/site-packages/xlsxwriter/worksheet.py:1170: UserWarning: Ignoring URL 'https://github.com/krishnaik06/OPENAI-API-Tutorials
In%20an%20API%20call,%20you%20can%20describe%20functions%20to%20gpt-3.5-turbo-0613%20and%20gpt-4-0613,%20and%20have%20the%20model%20intelligently%20choose%20to%20output%20a%20JSON%20object%20containing%20arguments%20to%20call%20those%20functions.%20The%20Chat%20Completions%20API%20does%20not%20call%20the%20function;%20instead,%20the%20model%20generates%20JSON%20that%20you%20can%20use%20to%20call%20the%20function%20in%20your%20code.

Support%20me%20by%20joining%20membership%20so%20that%20I%20can%20upload%20these%20kind%20of%20videos
https://www.youtube.com/channel/UCNU_lfiiWBdtULKOw6X0Dig/join

Check%20out%20our%20Full%20Stack%20Data%20Science%20Masters%20Course%20With%20Job%20Assistance
https://bit.ly/3UGeZQz

Data%20Science%20Projects:%20
https://www.youtube.com/watch?v=S_F_c9e2bz4&list=PLZoTAELRMXVPS-dOaVbAux22vzqdgoGhG

In [ ]:
# comments_dataframes = [SundasKhalid_comment_df, Annamonova_comment_df, LukeBarousse_comment_df, edrea_comment_df, RaneshGuruparan_comment_df, 
#             MohamedAlAssaal_comment_df, LoreSoWhat_comment_df, ThuVu_comment_df, DataWithMo_comment_df, DeenaGergis_comment_df, techTFQ_comment_df, 
#             LearnwithLukas_comment_df, AlexTheAnalyst_comment_df, codebasics_comment_df, MustafaOthman_comment_df, JustinShin_comment_df, KenJee_comment_df,
#             YoussefHosni_comment_df]


# # Concatenate the dataframes
# youtube_comments_data = pd.concat(comments_dataframes, ignore_index=True)
# youtube_comments_data

In [ ]:
# # Save the channel_stats DataFrame to a CSV file
# youtube_comments_data.to_excel('/Users/godzilla/Desktop/Selected/Project/youtube_comments_data_final.xlsx', index=False)
